In [117]:
import pandas as pd
import numpy as np
import math as m

In [118]:
dir_path = 'data/waypoints.csv'
df = pd.read_csv(dir_path)
wx = df['y'].values.tolist()
wy = df['x'].values.tolist()
wx = wx[20:50]
wy = wy[20:50]

# Vectorisation

In [119]:
def solve_1st_derivative(x, y):

    dx = np.ediff1d(x)
    dy = np.ediff1d(y)
    dx = np.concatenate((dx, [dx[0]]))
    dy = np.concatenate((dy, [dy[0]]))

    return dx, dy

In [120]:
%%capture
def calculate_yaw_vectorised(x, y):

    dx, dy = solve_1st_derivative(x, y)
        
    return np.arctan2(dy, dx)

def calculate_yaw_optimised(x, y):

    dx, dy = solve_1st_derivative(x, y)

    yaw = []

    yaw.append(m.atan2(dy[0], dx[0]))

    for i in range(1, len(x) - 1):
        d0 = np.array([dx[i], dy[i]])
        d0_hat = d0/np.linalg.norm(d0)

        d1 = np.array([dx[i-1], dy[i-1]])
        d1_hat = d1/np.linalg.norm(d1)

        d_bisect = d0_hat + d1_hat
        
        yaw.append(m.atan2(d_bisect[1], d_bisect[0]))

    yaw.append(m.atan2(dy[-1], dx[-1]))
        
    return yaw

def calculate_yaw(x, y):

    dx, dy = solve_1st_derivative(x, y)

    yaw = []

    for i in range(0, len(x)):
        yaw.append(np.arctan2(dy[i], dx[i]))

    return yaw

t_vectorised = %timeit -o calculate_yaw_vectorised(wx, wy)
t_optimised = %timeit -o calculate_yaw_optimised(wx, wy)
t_default = %timeit -o calculate_yaw(wx, wy)

In [121]:
print("\nVectorised test:\n{}".format(t_vectorised))
print("\n'Optimised' test:\n{}".format(t_optimised))
print("\nDefault test:\n{}".format(t_default))


Vectorised test:
13.8 µs ± 68.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

'Optimised' test:
373 µs ± 2.01 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Default test:
64.3 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Exponents

In [122]:
%%capture

def nat_pow(a, exp):

    for i in range(0, exp):
        a *= a

    return a

a = 7.6
exp = 3
t_manual = %timeit -o a*a
t_numpy = %timeit -o np.power(a, exp)
t_math = %timeit -o m.pow(a, exp)
t_optimised = %timeit -o nat_pow(a, exp)
t_standard = %timeit -o pow(a, exp)
t_default = %timeit -o a**exp

In [123]:
print("\nManual test:\n{}".format(t_manual))
print("\nNumpy test:\n{}".format(t_numpy))
print("\nMath test:\n{}".format(t_math))
print("\n'Optimised' test:\n{}".format(t_optimised))
print("\nStandard test:\n{}".format(t_standard))
print("\nDefault test:\n{}".format(t_default))


Manual test:
29 ns ± 0.289 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

Numpy test:
1.18 µs ± 29.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Math test:
93.5 ns ± 1.23 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

'Optimised' test:
226 ns ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Standard test:
84.8 ns ± 1.81 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

Default test:
60.8 ns ± 0.621 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# Path Curvature

In [124]:
%%capture

from libs.cubic_spline_interpolator import calculate_spline_curvature, generate_cubic_path

def calculate_curvature_vector(wx, wy):

    x, y = generate_cubic_path(wx, wy, 0.1)
    dp = solve_1st_derivative(x, y)
    delta_s = np.hypot(dp[0], dp[1])
    u_i = dp / delta_s

    du = solve_1st_derivative(u_i[0], u_i[1])
    abs_du = np.hypot(du[0], du[1])
    k_i = abs_du / delta_s

    return k_i

def calculate_curvature(wx, wy):

    x, y = generate_cubic_path(wx, wy, 0.1)
    dx, dy = solve_1st_derivative(x, y)
    ddx, ddy = solve_1st_derivative(dx, dy)

    return (ddy*dx - ddx*dy) / ((dx*dx + dy*dy)**(3/2))

t_vector = %timeit -o calculate_curvature_vector(wx, wy)
k_vector = calculate_curvature_vector(wx, wy).tolist()[:20]

t_spline = %timeit -o calculate_spline_curvature(wx, wy, 0.1)
k_spline = calculate_spline_curvature(wx, wy, 0.1)[:20]

t_default = %timeit -o calculate_curvature(wx, wy)
k_default = calculate_curvature(wx, wy).tolist()[:20]

In [125]:
print("\nVector test:\n{} \n{}".format(t_vector, k_vector))
print("\nSpline test:\n{} \n{}".format(t_spline, k_spline))
print("\nDefault test:\n{} \n{}".format(t_default, k_default))


Vector test:
4.79 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) 
[0.0071794594466529195, 0.014371994374315682, 0.02159222183660794, 0.02885455210393894, 0.03617306962996375, 0.04356140358782242, 0.051032584220446806, 0.05859888145617443, 0.06627162217067649, 0.0740609827063314, 0.08197575349652425, 0.09002307307734701, 0.09820812928166156, 0.10653382647688614, 0.11500041866865367, 0.12360511011479197, 0.13234162690684484, 0.1411997657250691, 0.15016492890792035, 0.1592176586500857]

Spline test:
7.68 ms ± 30.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) 
[-1.037259719017142e-16, 0.007179460134010003, 0.01437363303432191, 0.021597122355828228, 0.028864309868369735, 0.036189235354098834, 0.04358546550513043, 0.05106594815776471, 0.05864284823360333, 0.06632736184122506, 0.07412950515381349, 0.08205787495466292, 0.09011937817797741, 0.09831892841865345, 0.10665910830055914, 0.1151397978420971, 0.1237577706062772, 0.13250626153231723, 0.1413745129677447, 0

# Prepending

In [126]:
%%capture

def solve_1st_derivative_insert(x, y):

    a = np.array([0])
    dx = np.diff(x, append=a[0])
    dy = np.diff(y, append=a[0])

    return dx, dy

def solve_1st_derivative_concat(x, y):

    dx = np.ediff1d(x)
    dy = np.ediff1d(y)
    dx = np.concatenate((dx, [dx[0]]))
    dy = np.concatenate((dy, [dy[0]]))

    return dx, dy

t_insert = %timeit -o solve_1st_derivative_insert(wx, wy)
d = solve_1st_derivative_insert(wx, wy)
d_insert = d[0][:5]

t_concat = %timeit -o solve_1st_derivative_concat(wx, wy)
d = solve_1st_derivative_concat(wx, wy)
d_concat = d[0][:5]

In [127]:
print("\nInsert test:\n{}\n{}".format(t_insert, d_insert))
print("\nConcatenate test:\n{}\n{}".format(t_concat, d_concat))


Insert test:
29.2 µs ± 697 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
[2.15463259 1.21198083 0.73237982 0.06881949 0.05440933]

Concatenate test:
12.1 µs ± 74.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
[2.15463259 2.15463259 1.21198083 0.73237982 0.06881949]
